### Installation Instructions

Run the cell below to install the required libraries. Once the installation is complete, restart the session by going to **Runtime** and selecting **Restart session**.


In [2]:
from IPython.display import display, HTML

display(HTML('''
    <script>
    code_show = true;
    function code_toggle() {
        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show;
    }
    </script>
    <form action="javascript:code_toggle()">
        <input type="submit" value="Toggle Code">
    </form>
'''))


%cd /content
!GIT_LFS_SKIP_SMUDGE=1 git clone -b dev https://github.com/camenduru/InstantMesh

!pip install pytorch-lightning==2.1.2 gradio==3.50.2 einops omegaconf torchmetrics webdataset accelerate tensorboard
!pip install PyMCubes trimesh rembg transformers diffusers==0.20.2 bitsandbytes imageio[ffmpeg] xatlas plyfile
!pip install git+https://github.com/NVlabs/nvdiffrast jax==0.4.19 jaxlib==0.4.19 ninja
!pip install onnxruntime
!pip install huggingface_hub==0.25.2 --force-reinstall
!pip install jax==0.4.16 jaxlib==0.4.16 flax==0.7.4

/content
fatal: destination path 'InstantMesh' already exists and is not an empty directory.
  Cloning https://github.com/NVlabs/nvdiffrast to /tmp/pip-req-build-8bfno388
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast /tmp/pip-req-build-8bfno388
  Resolved https://github.com/NVlabs/nvdiffrast to commit 729261dc64c4241ea36efda84fbf532cc8b425b8
  Preparing metadata (setup.py) ... done
  Using cached jax-0.4.19-py3-none-any.whl.metadata (23 kB)
  Using cached jaxlib-0.4.19-cp311-cp311-manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached jax-0.4.19-py3-none-any.whl (1.7 MB)
Using cached jaxlib-0.4.19-cp311-cp311-manylinux2014_x86_64.whl (85.3 MB)
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.16
    Uninstalling jaxlib-0.4.16:
      Successfully uninstalled jaxlib-0.4.16
  Attempting uninstall: jax
    Found existing installation: jax 0.4.16
    Uninstalling jax-0.4.16:
      Successfully uninstalled jax-0.4.16
  U

  Using cached jax-0.4.16-py3-none-any.whl.metadata (29 kB)
  Using cached jaxlib-0.4.16-cp311-cp311-manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached jax-0.4.16-py3-none-any.whl (1.6 MB)
Using cached jaxlib-0.4.16-cp311-cp311-manylinux2014_x86_64.whl (84.5 MB)
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.19
    Uninstalling jaxlib-0.4.19:
      Successfully uninstalled jaxlib-0.4.19
  Attempting uninstall: jax
    Found existing installation: jax 0.4.19
    Uninstalling jax-0.4.19:
      Successfully uninstalled jax-0.4.19


# Download the pretrained model

In [3]:
%cd /content/InstantMesh
import torch
import numpy as np
import rembg
from PIL import Image
from pytorch_lightning import seed_everything
from einops import rearrange
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
from huggingface_hub import hf_hub_download
from src.utils.infer_util import remove_background, resize_foreground

# ---------------------------
# Download Diffusion Pipeline
# ---------------------------
print("Downloading diffusion pipeline 'sudo-ai/zero123plus-v1.2'...")
pipeline = DiffusionPipeline.from_pretrained(
    "sudo-ai/zero123plus-v1.2",
    custom_pipeline="zero123plus",
    torch_dtype=torch.float16,
)
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipeline.scheduler.config, timestep_spacing='trailing'
)
print("Diffusion pipeline downloaded successfully.\n")

# ---------------------------
# Download UNet Checkpoint
# ---------------------------
print("Downloading UNet checkpoint...")
unet_ckpt_path = hf_hub_download(
    repo_id="TencentARC/InstantMesh",
    filename="diffusion_pytorch_model.bin",
    repo_type="model"
)
print(f"UNet checkpoint downloaded at:\n  {unet_ckpt_path}\n")

# Load and apply the UNet state_dict
state_dict = torch.load(unet_ckpt_path, map_location="cpu")
pipeline.unet.load_state_dict(state_dict, strict=True)

# ---------------------------
# Download InstantMesh Checkpoint
# ---------------------------
print("Downloading InstantMesh model checkpoint...")
mesh_model_ckpt_path = hf_hub_download(
    repo_id="TencentARC/InstantMesh",
    filename="instant_mesh_base.ckpt",
    repo_type="model"
)
print(f"InstantMesh model checkpoint downloaded at:\n  {mesh_model_ckpt_path}\n")

print("All pretrained models have been downloaded successfully!")


/content/InstantMesh


Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x7972af173920>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/llvmlite/binding/executionengine.py", line 178, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):

KeyboardInterrupt: 


RuntimeError: no compiled object yet for <Library 'boxfilter_rows_valid' at 0x7971ae6a25d0>

# Gradio app
### Installation Instructions

Run the cell below to install the required libraries. Once the installation is complete, restart the session by going to **Runtime** and selecting **Restart session**.

### Server Port Adjustment

If you encounter any errors in the code below related to the server port, adjust the `server_port` as follows:

- **Example:**
  ```python
  server_port = 43538
  ```
  - change it to

  ```python
  server_port = 43548
  ```


In [ ]:
import os
import imageio
import numpy as np
import torch
import rembg
from PIL import Image
from torchvision.transforms import v2
from pytorch_lightning import seed_everything
from omegaconf import OmegaConf
from einops import rearrange
from tqdm import tqdm
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
import tempfile
from huggingface_hub import hf_hub_download
import gc
import gradio as gr

#############################################
# MemoryOptimizedInstantMesh (Diffusion/Reconstruction)
#############################################
class MemoryOptimizedInstantMesh:
    def __init__(self, config_path):
        self.config = OmegaConf.load(config_path)
        self.config_name = os.path.basename(config_path).replace('.yaml', '')
        self.model_config = self.config.model_config
        self.infer_config = self.config.infer_config
        self.is_flexicubes = True if self.config_name.startswith('instant-mesh') else False
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.pipeline = None

    def load_pipeline(self):
        """Load the diffusion pipeline with memory management"""
        if self.pipeline is not None:
            return
        self.pipeline = DiffusionPipeline.from_pretrained(
            "sudo-ai/zero123plus-v1.2",
            custom_pipeline="zero123plus",
            torch_dtype=torch.float16,
        )
        self.pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
            self.pipeline.scheduler.config, timestep_spacing='trailing'
        )
        unet_ckpt_path = hf_hub_download(
            repo_id="TencentARC/InstantMesh",
            filename="diffusion_pytorch_model.bin",
            repo_type="model"
        )
        state_dict = torch.load(unet_ckpt_path, map_location='cpu')
        self.pipeline.unet.load_state_dict(state_dict, strict=True)
        self.pipeline = self.pipeline.to(self.device)

    def unload_pipeline(self):
        """Unload the pipeline and clear GPU memory"""
        if self.pipeline is not None:
            self.pipeline = self.pipeline.to('cpu')
            del self.pipeline
            self.pipeline = None
            torch.cuda.empty_cache()
            gc.collect()

    def load_model(self):
        """Load the reconstruction model with memory management"""
        if self.model is not None:
            return
        from src.utils.train_util import instantiate_from_config
        model_ckpt_path = hf_hub_download(
            repo_id="TencentARC/InstantMesh",
            filename="instant_mesh_base.ckpt",
            repo_type="model"
        )
        self.model = instantiate_from_config(self.model_config)
        state_dict = torch.load(model_ckpt_path, map_location='cpu')['state_dict']
        state_dict = {k[14:]: v for k, v in state_dict.items()
                      if k.startswith('lrm_generator.') and 'source_camera' not in k}
        self.model.load_state_dict(state_dict, strict=True)
        self.model = self.model.to(self.device)
        if self.is_flexicubes:
            self.model.init_flexicubes_geometry(self.device, fovy=30.0)
        self.model = self.model.eval()

    def unload_model(self):
        """Unload the reconstruction model and clear GPU memory"""
        if self.model is not None:
            self.model = self.model.to('cpu')
            del self.model
            self.model = None
            torch.cuda.empty_cache()
            gc.collect()

    def remove_background(self, image, session):
        """Remove background from image using rembg"""
        if isinstance(image, Image.Image):
            image = np.array(image)
        return Image.fromarray(rembg.remove(image, session=session))

    def resize_foreground(self, image, scale=0.85):
        """Resize foreground content"""
        image_arr = np.array(image)
        if image_arr.shape[2] == 4:
            mask = image_arr[:, :, 3] > 0
            coords = np.where(mask)
            if len(coords[0]) == 0:
                return image
            y_min, y_max = coords[0].min(), coords[0].max()
            x_min, x_max = coords[1].min(), coords[1].max()
            size = max(y_max - y_min, x_max - x_min)
            target_size = int(min(image.size) * scale)
            if size > target_size:
                scale_factor = target_size / size
                new_h = int((y_max - y_min) * scale_factor)
                new_w = int((x_max - x_min) * scale_factor)
                cropped = image.crop((x_min, y_min, x_max, y_max))
                resized = cropped.resize((new_w, new_h), Image.Resampling.LANCZOS)
                new_image = Image.new('RGBA', image.size, (0, 0, 0, 0))
                paste_x = (image.size[0] - new_w) // 2
                paste_y = (image.size[1] - new_h) // 2
                new_image.paste(resized, (paste_x, paste_y))
                return new_image
        return image

    def generate_mvs(self, input_image, sample_steps, sample_seed):
        """Generate multi-view images with memory cleanup"""
        self.load_pipeline()
        seed_everything(sample_seed)
        try:
            generator = torch.Generator(device=self.device)
            z123_image = self.pipeline(
                input_image,
                num_inference_steps=sample_steps,
                generator=generator,
            ).images[0]
            show_image = np.asarray(z123_image, dtype=np.uint8)
            show_image = torch.from_numpy(show_image)
            show_image = rearrange(show_image, '(n h) (m w) c -> (n m) h w c', n=3, m=2)
            show_image = rearrange(show_image, '(n m) h w c -> (n h) (m w) c', n=2, m=3)
            show_image = Image.fromarray(show_image.numpy())
            return z123_image, show_image
        finally:
            self.unload_pipeline()

    def get_render_cameras(self, batch_size=1, M=120, radius=2.5, elevation=10.0):
        """Get rendering camera parameters"""
        from src.utils.camera_util import (
            FOV_to_intrinsics,
            get_circular_camera_poses,
        )
        c2ws = get_circular_camera_poses(M=M, radius=radius, elevation=elevation)
        if self.is_flexicubes:
            cameras = torch.linalg.inv(c2ws)
            cameras = cameras.unsqueeze(0).repeat(batch_size, 1, 1, 1)
        else:
            extrinsics = c2ws.flatten(-2)
            intrinsics = FOV_to_intrinsics(30.0).unsqueeze(0).repeat(M, 1, 1).float().flatten(-2)
            cameras = torch.cat([extrinsics, intrinsics], dim=-1)
            cameras = cameras.unsqueeze(0).repeat(batch_size, 1, 1)
        return cameras.to(self.device)

    def make3d(self, images):
        """Generate 3D model with memory optimization"""
        self.load_model()
        try:
            images = np.asarray(images, dtype=np.float32) / 255.0
            images = torch.from_numpy(images).permute(2, 0, 1).contiguous().float()
            images = rearrange(images, 'c (n h) (m w) -> (n m) c h w', n=3, m=2)
            from src.utils.camera_util import get_zero123plus_input_cameras
            input_cameras = get_zero123plus_input_cameras(
                batch_size=1, radius=4.0).to(self.device)
            render_cameras = self.get_render_cameras(
                batch_size=1, radius=4.5, elevation=20.0).to(self.device)
            images = images.unsqueeze(0).to(self.device)
            images = v2.functional.resize(
                images, (320, 320),
                interpolation=3,
                antialias=True
            ).clamp(0, 1)
            with torch.no_grad():
                planes = self.model.forward_planes(images, input_cameras)
                frames = self.render_frames(render_cameras, planes)
            video_path = self.save_video(frames[0])
            mesh_path = self.generate_mesh(planes)
            return video_path, mesh_path
        finally:
            del images, input_cameras, render_cameras, planes, frames
            torch.cuda.empty_cache()
            gc.collect()
            self.unload_model()

    def render_frames(self, render_cameras, planes):
        """Render frames with chunked processing"""
        chunk_size = 20 if self.is_flexicubes else 1
        render_size = 384
        frames = []
        for i in tqdm(range(0, render_cameras.shape[1], chunk_size)):
            if self.is_flexicubes:
                frame = self.model.forward_geometry(
                    planes,
                    render_cameras[:, i:i+chunk_size],
                    render_size=render_size,
                )['img']
            else:
                frame = self.model.synthesizer(
                    planes,
                    cameras=render_cameras[:, i:i+chunk_size],
                    render_size=render_size,
                )['images_rgb']
            frames.append(frame)
            if i + chunk_size < render_cameras.shape[1]:
                torch.cuda.empty_cache()
        return torch.cat(frames, dim=1)

    def generate_mesh(self, planes):
        """Generate mesh with memory optimization"""
        from src.utils.mesh_util import save_obj
        mesh_path = tempfile.NamedTemporaryFile(suffix=".obj", delete=False).name
        with torch.no_grad():
            mesh_out = self.model.extract_mesh(
                planes,
                use_texture_map=False,
                **self.infer_config,
            )
            vertices, faces, vertex_colors = mesh_out
            vertices = vertices[:, [1, 2, 0]]
            vertices[:, -1] *= -1
            faces = faces[:, [2, 1, 0]]
            save_obj(vertices, faces, vertex_colors, mesh_path)
        return mesh_path

    def save_video(self, frames, fps=30):
        """Save video frames with memory cleanup"""
        video_path = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False).name
        frames_np = []
        for i in range(frames.shape[0]):
            frame = (frames[i].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            frames_np.append(frame)
        imageio.mimwrite(video_path, np.stack(frames_np), fps=fps, codec='h264')
        del frames_np
        return video_path


#############################################
# Gradio Demo Creation
#############################################
def create_demo(instant_mesh):
    _HEADER_ = '''
    3D Gen: Image to 3D Generation
    '''

    def check_input_image(input_image):
        if input_image is None:
            raise gr.Error("No image uploaded!")

    def process_input(input_image, do_remove_background):
        if do_remove_background:
            rembg_session = rembg.new_session()
            input_image = instant_mesh.remove_background(input_image, rembg_session)
        input_image = instant_mesh.resize_foreground(input_image, 0.85)
        return input_image

    with gr.Blocks() as demo:
        gr.Markdown(_HEADER_)
        with gr.Row(variant="panel"):
            with gr.Column():
                with gr.Row():
                    input_image = gr.Image(
                        label="Input Image",
                        image_mode="RGBA",
                        width=256,
                        height=256,
                        type="pil",
                        elem_id="content_image",
                    )
                    processed_image = gr.Image(
                        label="Processed Image",
                        image_mode="RGBA",
                        width=256,
                        height=256,
                        type="pil",
                        interactive=False
                    )
                with gr.Row():
                    with gr.Group():
                        do_remove_background = gr.Checkbox(
                            label="Remove Background",
                            value=True
                        )
                        sample_seed = gr.Number(
                            value=42,
                            label="Seed Value",
                            precision=0
                        )
                        sample_steps = gr.Slider(
                            label="Sample Steps",
                            minimum=30,
                            maximum=75,
                            value=75,
                            step=5
                        )
                with gr.Row():
                    submit = gr.Button(
                        "Generate",
                        elem_id="generate",
                        variant="primary"
                    )

            with gr.Column():
                with gr.Row():
                    with gr.Column():
                        mv_show_images = gr.Image(
                            label="Generated Multi-views",
                            type="pil",
                            width=379,
                            interactive=False
                        )
                    with gr.Column():
                        output_video = gr.Video(
                            label="Video",
                            format="mp4",
                            width=379,
                            autoplay=True,
                            interactive=False
                        )
                with gr.Row():
                    output_model_obj = gr.Model3D(
                        label="Output Model (OBJ Format)",
                        width=768,
                        interactive=False,
                    )

        # State for storing intermediate results
        mv_images = gr.State()

        # Main generation pipeline
        submit.click(
            fn=check_input_image,
            inputs=[input_image]
        ).success(
            fn=process_input,
            inputs=[input_image, do_remove_background],
            outputs=[processed_image],
        ).success(
            fn=lambda x, y, z: instant_mesh.generate_mvs(x, y, z),
            inputs=[processed_image, sample_steps, sample_seed],
            outputs=[mv_images, mv_show_images],
        ).success(
            fn=lambda x: instant_mesh.make3d(x),
            inputs=[mv_images],
            outputs=[output_video, output_model_obj]
        )

    return demo


#############################################
# Main
#############################################
def main():
    # Initialize InstantMesh (ensure your config file path is correct)
    instant_mesh = MemoryOptimizedInstantMesh(config_path='configs/instant-mesh-base.yaml')
    # Create and launch the Gradio demo
    demo = create_demo(instant_mesh)
    demo.queue(max_size=10)
    demo.launch(
        server_name="0.0.0.0",
        server_port=43548,
        share=True
    )


if __name__ == "__main__":
    main()